In [6]:
import json
import time
import random
from collections import defaultdict
from tqdm import tqdm
import google.generativeai as genai

genai.configure(api_key="<paste_your_gemini_api_key_here>")

Mounted at /content/drive


In [7]:
def build_prompt(question, shots=[]):

    prompt_instructions = (
        "You are an expert at translating natural language questions into SQL queries for the ATIS flight database.\n"
        "Use the provided schemas and follow these instructions carefully:\n"
        "- Always query the FLIGHT table to retrieve FLIGHT_ID using SELECT DISTINCT.\n"
        "- Join through CITY, AIRPORT_SERVICE, DATE_DAY, and DAYS to map city names and day names correctly.\n"
        "- Use placeholders like city_name0, city_name1, day_number0, month_number0, year0 where appropriate.\n"
        "- Use subqueries rather than join operations.\n"
        "- DO NOT hardcode values like FLIGHT_DAYS = 'MONDAY' or FLIGHT_DAYS = 2 — instead, JOIN with the DAYS and DATE_DAY tables.\n"
        "- DO NOT use constants like 'MONDAY', 'TUESDAY', or numeric day codes directly — always use placeholders + joins.\n"
        "- DO NOT use nested SELECTs unless strictly necessary.\n"
        "- DO NOT use fields not present in the schema (e.g., FLIGHT_SOURCE, FLIGHT_DESTINATION).\n"
        "- DO NOT use irrelevant joins like FLIGHT_STOP, AIRLINE, or CITYalias2 unless the question explicitly requires it (e.g., intermediate stops, airlines).\n"
        "- DO NOT invent or reference fields like DATE_DAYalias0.DATE_DAY — only use columns present in the provided schema.\n"
        "- DO NOT query FARE or use FLIGHT_FARE unless the question explicitly asks about prices or cost-related details.\n"
        "- DO NOT include any newline character in the SQL query generated.\n"
        "- Use CITY.CITY_NAME joined via AIRPORT_SERVICE and FLIGHT for city-based queries.\n"
        "- Always use FLIGHT.FROM_AIRPORT and FLIGHT.TO_AIRPORT.\n"
        "- Always map FLIGHT.FLIGHT_DAYS to DAYS.DAY_NAME via DATE_DAY.\n"
        "- Always format the FROM and WHERE clauses clearly and consistently with proper alias usage.\n"
        "- Always complete the parentheses.\n"
        "- Always add ';' at the end of the SQL query generated.\n"
        "- Return only the SQL query. Do not include explanations, comments, or additional output.\n\n"
    )

    formatted_shots = []
    for ex in shots:
        formatted_shots.append(f"Q: {ex['question']} => {ex['sql']}")

    return prompt_instructions + "\n".join(formatted_shots) + f"\nQ: {question} =>"

In [8]:
def generate_sql(question, shots=[], retries=20, base_wait=2):

    prompt = build_prompt(question, shots)
    for attempt in range(retries):
        try:
            model = genai.GenerativeModel("gemini-1.5-pro")
            response = model.generate_content(prompt)
            return response.text.strip()

        except Exception as e:
            wait = base_wait * (attempt + 1)
            print(f"Error: {e}. Waiting {wait}s...")
            time.sleep(wait)
    raise RuntimeError("Runtime Error: Failed to generate after multiple retries.")

def load_shot_examples(examples, shots):

    shot_examples = []
    for example in examples:
        sql_list = example.get("sql", [])
        if not sql_list:
            continue
        sql = min(sql_list, key=len).replace("\n", " ")
        for sent in example.get("sentences", []):
            if sent.get("question-split") not in ["train", "dev"]:
                continue
            shot_examples.append({"question": sent["text"], "sql": sql})
    # random.seed(5046)
    random.shuffle(shot_examples)
    return shot_examples[:shots]

def get_eval_examples(examples, split):

    eval_examples = []
    for example in examples:
        sql_list = example.get("sql", [])
        if not sql_list:
            continue
        sql = min(sql_list, key=len).replace("\n", " ")
        for sent in example.get("sentences", []):
            if sent.get("question-split") == split:
                eval_examples.append({"text": sent["text"], "final_sql": sql})
    return eval_examples

def evaluate(eval_examples, shots):

    correct = 0
    for example in tqdm(eval_examples):
        question = example["text"]
        final = example["final_sql"]
        predicted = generate_sql(question, shots)
        print("\nQuestion:", question)
        print("Predicted SQL:", predicted)
        print("Correct SQL:", final)
        if predicted == final:
            print("CORRECT!!!\n")
            correct += 1
        else:
            print("INCORRECT\n")

    print(f"Accuracy: {correct}/{len(eval_examples)} = {(correct/len(eval_examples)):.5f}")

In [9]:
with open("atis.json") as dataset:
    examples = json.load(dataset)

In [10]:
print("Evaluating the LLM Gemini 1.5 Pro on ATIS by \"question-split\" and 0-shot training (no examples)...")
evaluate(get_eval_examples(examples, split="test"), load_shot_examples(examples, 0))

Evaluating the LLM Gemini 1.5 Pro on ATIS by "question-split" and 0-shot training (no examples)...


  0%|          | 1/447 [00:03<29:31,  3.97s/it]


Question: i need a flight from city_name1 to city_name0 on monday
Predicted SQL: SELECT DISTINCT FLIGHT.FLIGHT_ID FROM FLIGHT WHERE FLIGHT.FROM_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'city_name1')) AND FLIGHT.TO_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'city_name0')) AND FLIGHT.FLIGHT_DAYS IN (SELECT DATE_DAY.DAY_OF_WEEK FROM DATE_DAY WHERE DATE_DAY.DAY_NAME = (SELECT DAYS.DAY_CODE FROM DAYS WHERE DAYS.DAY_NAME = 'MONDAY'));
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , DATE_DAY AS DATE_DAYalias0 , DAYS AS DAYSalias0 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYa

  0%|          | 2/447 [00:06<23:31,  3.17s/it]


Question: list flights from city_name1 to city_name0 on wednesday
Predicted SQL: SELECT DISTINCT FLIGHT.FLIGHT_ID FROM FLIGHT WHERE FLIGHT.FROM_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'city_name1')) AND FLIGHT.TO_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'city_name0')) AND FLIGHT.FLIGHT_DAYS IN (SELECT DATE_DAY.DAY_OF_WEEK FROM DATE_DAY WHERE DATE_DAY.DAY_NAME = 'Wednesday');
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , DATE_DAY AS DATE_DAYalias0 , DAYS AS DAYSalias0 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "city_name0" AND DATE_DAYalias0.DA

...


Question: is there one airline that flies from city_name1 to city_name2 milwaukee to city_name0 and from city_name0 to city_name1
Predicted SQL: SELECT DISTINCT FLIGHT.FLIGHT_ID FROM FLIGHT WHERE FLIGHT.FROM_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'city_name1')) AND FLIGHT.TO_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'city_name2')) INTERSECT SELECT DISTINCT FLIGHT.FLIGHT_ID FROM FLIGHT WHERE FLIGHT.FROM_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'MILWAUKEE')) AND FLIGHT.TO_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'city_

100%|█████████▉| 446/447 [19:46<00:04,  4.04s/it]


Question: i'd like to book two flights to city_name0
Predicted SQL: SELECT DISTINCT FLIGHT.FLIGHT_ID FROM FLIGHT WHERE FLIGHT.TO_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'city_name0'));
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM FLIGHT AS FLIGHTalias0 WHERE FLIGHTalias0.TO_AIRPORT IN ( SELECT AIRPORT_SERVICEalias0.AIRPORT_CODE FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 WHERE AIRPORT_SERVICEalias0.CITY_CODE IN ( SELECT CITYalias0.CITY_CODE FROM CITY AS CITYalias0 WHERE CITYalias0.CITY_NAME = "city_name0" ) ) ;
Incorrect


100%|██████████| 447/447 [19:48<00:00,  2.66s/it]


Question: find me a flight from city_name0 to any airport in the city_name1 area
Predicted SQL: SELECT DISTINCT FLIGHT.FLIGHT_ID FROM FLIGHT WHERE FLIGHT.FROM_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'city_name0')) AND FLIGHT.TO_AIRPORT IN (SELECT AIRPORT_SERVICE.AIRPORT_CODE FROM AIRPORT_SERVICE WHERE AIRPORT_SERVICE.CITY_CODE IN (SELECT CITY.CITY_CODE FROM CITY WHERE CITY.CITY_NAME = 'city_name1'));
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM FLIGHT AS FLIGHTalias0 WHERE ( FLIGHTalias0.FROM_AIRPORT IN ( SELECT AIRPORT_SERVICEalias0.AIRPORT_CODE FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 WHERE AIRPORT_SERVICEalias0.CITY_CODE IN ( SELECT CITYalias0.CITY_CODE FROM CITY AS CITYalias0 WHERE CITYalias0.CITY_NAME = "city_name0" ) ) AND FLIGHTalias0.TO_AIRPORT IN ( SELECT AIRPORTalias0.AIRPORT_CODE FROM AIRPORT AS AIRPORTalias0 WHERE AIRPORTalias0.AIRPORT_CO

In [ ]:
print("Evaluating the LLM Gemini 1.5 Pro on ATIS by \"question-split\" and 5-shot training (few examples)...")
evaluate(get_eval_examples(examples, split="test"), load_shot_examples(examples, 5))

In [11]:
print("Evaluating the LLM Gemini 1.5 Pro on ATIS by \"question-split\" and 40-shot training (many examples)...")
evaluate(get_eval_examples(examples, split="test"), load_shot_examples(examples, 40))

Evaluating LLM Gemini 1.5 Pro on ATIS by "question-split" and 40-shot training (many examples)...


  0%|          | 1/447 [00:04<37:05,  4.99s/it]


Question: i need a flight from city_name1 to city_name0 on monday
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , DATE_DAY AS DATE_DAYalias0 , DAYS AS DAYSalias0 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "city_name0" AND DATE_DAYalias0.DAY_NUMBER = day_number0 AND DATE_DAYalias0.MONTH_NUMBER = month_number0 AND DATE_DAYalias0.YEAR = year0 AND DAYSalias0.DAY_NAME = DATE_DAYalias0.DAY_NAME AND FLIGHTalias0.FLIGHT_DAYS = DAYSalias0.DAYS_CODE AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ) AND CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "city_name1" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_S

  0%|          | 2/447 [00:09<33:23,  4.50s/it]


Question: list flights from city_name1 to city_name0 on wednesday
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , DATE_DAY AS DATE_DAYalias0 , DAYS AS DAYSalias0 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "city_name0" AND DATE_DAYalias0.DAY_NUMBER = day_number0 AND DATE_DAYalias0.MONTH_NUMBER = month_number0 AND DATE_DAYalias0.YEAR = year0 AND DAYSalias0.DAY_NAME = DATE_DAYalias0.DAY_NAME AND FLIGHTalias0.FLIGHT_DAYS = DAYSalias0.DAYS_CODE AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ) AND CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "city_name1" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_S

...


Question: is there one airline that flies from city_name1 to city_name2 milwaukee to city_name0 and from city_name0 to city_name1
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias2 , CITY AS CITYalias0 , CITY AS CITYalias1 , CITY AS CITYalias2 , FLIGHT AS FLIGHTalias0 WHERE ( CITYalias2.CITY_CODE = AIRPORT_SERVICEalias2.CITY_CODE AND CITYalias2.CITY_NAME = "city_name0" AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias2.AIRPORT_CODE ) AND CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "city_name1" AND CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "city_name2" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ;
Correct SQL: SELECT DISTINCT AIRLINEalias0.AIRLINE_CODE FROM AIRLINE AS AIRLINEalias0 WHERE

100%|█████████▉| 446/447 [26:22<00:04,  4.05s/it]


Question: i'd like to book two flights to city_name0
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , CITY AS CITYalias0 , FLIGHT AS FLIGHTalias0 WHERE CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "city_name0" AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM FLIGHT AS FLIGHTalias0 WHERE FLIGHTalias0.TO_AIRPORT IN ( SELECT AIRPORT_SERVICEalias0.AIRPORT_CODE FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 WHERE AIRPORT_SERVICEalias0.CITY_CODE IN ( SELECT CITYalias0.CITY_CODE FROM CITY AS CITYalias0 WHERE CITYalias0.CITY_NAME = "city_name0" ) ) ;
Incorrect


100%|██████████| 447/447 [26:25<00:00,  3.55s/it]


Question: find me a flight from city_name0 to any airport in the city_name1 area
Predicted SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 , AIRPORT_SERVICE AS AIRPORT_SERVICEalias1 , CITY AS CITYalias0 , CITY AS CITYalias1 , FLIGHT AS FLIGHTalias0 WHERE CITYalias0.CITY_CODE = AIRPORT_SERVICEalias0.CITY_CODE AND CITYalias0.CITY_NAME = "city_name0" AND CITYalias1.CITY_CODE = AIRPORT_SERVICEalias1.CITY_CODE AND CITYalias1.CITY_NAME = "city_name1" AND FLIGHTalias0.FROM_AIRPORT = AIRPORT_SERVICEalias0.AIRPORT_CODE AND FLIGHTalias0.TO_AIRPORT = AIRPORT_SERVICEalias1.AIRPORT_CODE ;
Correct SQL: SELECT DISTINCT FLIGHTalias0.FLIGHT_ID FROM FLIGHT AS FLIGHTalias0 WHERE ( FLIGHTalias0.FROM_AIRPORT IN ( SELECT AIRPORT_SERVICEalias0.AIRPORT_CODE FROM AIRPORT_SERVICE AS AIRPORT_SERVICEalias0 WHERE AIRPORT_SERVICEalias0.CITY_CODE IN ( SELECT CITYalias0.CITY_CODE FROM CITY AS CITYalias0 WHERE CITYalias0.CITY_NAME = "city_name0" ) ) AND FLIGHTalias0.TO_AIRPOR